In [12]:
import requests
import cv2
import pandas as pd
import csv
from bs4 import BeautifulSoup
import unicodedata
import mysql.connector
import ipynb
from ipynb.fs.full.SqlQuery import brandAdd
from ipynb.fs.full.SqlQuery import categorieAdd
from ipynb.fs.full.SqlQuery import AddProduct
from ipynb.fs.full.SqlQuery import getCagId
from ipynb.fs.full.SqlQuery import getBrandId

In [13]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="scraping_data"
)

In [14]:
def spacenet_links():
    list=[]
    web = requests.get('https://spacenet.tn/')
    parse = BeautifulSoup(web.content, 'html.parser')
    head= parse.find_all('div', class_='title title_font')
    for h in head:
        try:
            info= h.find('a').text.strip()
            link= h.find('a').get('href')
        except:
            info= 'None'
            link='None'
        if(info != 'None' ):
            space_dict={
                'info':info,
                'link':link
            }
            list.append(space_dict)
    return list

In [15]:
def priceConv(price):
    c=''
    for p in price:
        if(p.isdigit()):
            c=c+p
    return int(c)

In [16]:
def space_det(url):
    text = requests.get(url).text
    soup = BeautifulSoup(text, 'lxml')
    try:
        name= soup.find('h1', class_ ='h1').text.replace('|','')
    except:
        name = 'Unknown'
    try:
        brand = soup.find('div', class_='product-manufacturer').a.img.get('alt')
        brand_logo = soup.find('div', class_='product-manufacturer').a.img.get('src')
    except:
        brand = 'Unknown'
        brand_logo = 'None'
    try:
        price= soup.find('div', class_ ='current-price').span.text
        price = priceConv(price)
    except:
        price = 0
    try:
        stock = soup.find('div', class_ = 'product-quantities').label.text
    except:
        stock = 'épuise'
    try:
        sku = soup.find('div', class_ = 'product-reference').span.text.strip()
    except:
        sku = 'None'
    try:    
        des = soup.find('div', class_ = 'product-des').p.text.strip()
    except:
        des='None'
    article ={
          'name':name,
          'brand': brand,
          'brand_logo':brand_logo,
          'price': price,
          'stock': stock,
          'sku': sku,
          'des': des,
      }
    return article

In [17]:
def next_url(url, x):
    if(x == 1):
        url=url+'?page='+str(x)
    else:
        url=url[:-1]
        if(x>10):
            url=url[:-1]
        url=url+str(x)
    web = requests.get(url)
    parse = BeautifulSoup(web.content, 'html.parser')
    if(parse.find('section', class_='page-content page-not-found')):
        return False  

    return url

In [ ]:
# mycursor = mydb.cursor()
list = spacenet_links()
for l in list:
    info = l['info']
    link = l['link']
    categorieAdd(info, link, 23)
    supCatId=getCagId(info,23)
    web = requests.get(link)
    parse = BeautifulSoup(web.content, 'html.parser')
    for x in range(1, 31):
        link= next_url(link,x)
        print(link)
        if(not(link == False)):
            web = requests.get(link)
            parse = BeautifulSoup(web.content, 'html.parser')
            card= parse.find_all('div', class_='item col-xs-12 col-sm-6 col-md-6 col-lg-3')
            for c in card:
                img= c.find('span', class_='cover_image').img['src']
                det_link= c.find('div', class_='product_name').a['href']
                det=space_det(det_link)
                brandAdd(det['brand'], det['brand_logo'], 23)
                supBrandId=getBrandId(det['brand'], 23)
                prodId=AddProduct(det['name'], det_link, det['sku'], det['des'], det['price'], det['stock'], 23, supBrandId, supCatId, img)
        else:
            break  

https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=1
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=2
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=3
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=4
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=5
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=6
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=7
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=8
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=9
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=10
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=11
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=12
https://spacenet.tn/categorie/18-vente-ordinateur-portable-tunisie?page=13
https://spacenet.tn/categorie/18-v